In [20]:
from langchain.retrievers import ParentDocumentRetriever
from langchain_chroma import Chroma
from langchain.storage import InMemoryStore
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import pickle
from langchain_huggingface import HuggingFaceEmbeddings
import os
import keys
import pandas as pd
os.environ["OPENAI_API_KEY"] = keys.OPENAI_API_KEY


llm = ChatOpenAI(model="gpt-3.5-turbo-0125", max_retries=50)
embeddings = HuggingFaceEmbeddings(model_name="avsolatorio/GIST-Embedding-v0")

c:\Users\Raunaq\Desktop\rag_eval\.rag\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
# url = "https://docs.streamlit.io/"
# loader = RecursiveUrlLoader(url=url, max_depth=5, extractor=lambda x: Soup(x, "html.parser").text)
# docs = loader.load()
# len(docs)
# for i in docs:
#     if "content_type" in i.metadata:
#         del i.metadata["content_type"]
#     if "language" in i.metadata:
#         del i.metadata["language"]
#     if "description" in i.metadata:
# del i.metadata["description"]
# pickle.dump(docs, open("docs", "wb"))

docs = pickle.load(open("docs", 'rb'))

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")
# db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

In [ ]:
# docs = docs[:39]

In [ ]:
from ragas.run_config import RunConfig
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from tqdm import tqdm
testset = []

config = RunConfig(max_retries=50, max_wait=30)

for i in tqdm(range(20, len(docs), 10)):
    generator = TestsetGenerator.from_langchain(llm, llm, embeddings)
    testset += [generator.generate_with_langchain_docs(docs[i:i+10], test_size=10, distributions={
        simple: 0.1, reasoning: 0.3, multi_context: 0.3, conditional: 0.3}, is_async=False, raise_exceptions=False, with_debugging_logs=True, run_config=config).to_pandas()]
    testset[-1].to_csv(f"{i+448795792}.csv", index=None)

pd.concat(testset).to_csv("questions.csv", index=None)

In [1]:
# vectorstore = Chroma(
#     collection_name="full_documents", embedding_function=embeddings, persist_directory="./parent"
# )
# # The storage layer for the parent documents
# store = InMemoryStore()
# retriever = ParentDocumentRetriever(
#     vectorstore=vectorstore,
#     docstore=store,
#     child_splitter=RecursiveCharacterTextSplitter(chunk_size=400),
#     parent_splitter=RecursiveCharacterTextSplitter(chunk_size=1000)

# )

# retriever.add_documents(docs)


In [2]:
from rag import setup
from langchain.schema import HumanMessage, AIMessage
chain, rag_chain = setup()


In [3]:
messages = []
prompt = "What is the function decorator used to cache functions that return data in Streamlit?"
messages+= [{"role": "user", "content": prompt}]

rag_context = rag_chain.invoke(prompt)
history = [HumanMessage(content=i["content"]+"\n") if i["role"] == "user" else AIMessage(
    content=i["content"]+"\n") for i in messages]

response = chain.invoke({"rag_context": rag_context, "chat_history": history})

response

'To cache functions that return data in Streamlit, you can use either @st.cache_data or @st.cache_resource decorators. These decorators help store the results of slow function calls, making the app faster by running the function only once. By decorating the function with @st.cache_data, Streamlit checks input parameters and function code to determine if the cached value can be used.'